# base

In [ ]:
//// test

open testing

## execution

### emit

In [ ]:
inl emit forall t. (x : t) : t =
    $'!x '

### emit_unit

In [ ]:
inl emit_unit forall t. (x : t) : () =
    $'!x '

### use

In [ ]:
inl use forall t. (x : t) : t =
    $'use !x = !x ' : ()
    $'!x '

## type

### unit

In [ ]:
nominal unit = $'unit'

## target

### backend_switch

In [ ]:
inl backend_switch forall t. x : t =
    real
        inl backend key : t =
            inl s = real_core.string_lit_to_symbol key
            real_core.record_type_try_find `(`x) s
                (forall v'. => (x s) ())
                (fun () => $'' : t)
        !!!!BackendSwitch (
            ("Gleam", backend "Gleam"),
            ("Fsharp", backend "Fsharp"),
            ("Python", backend "Python"),
            ("Cuda", backend "Cuda")
        )

### target_runtime

In [ ]:
union target_runtime =
    | Native
    | Wasm
    | Contract

### target

In [ ]:
union target =
    | Gleam : target_runtime
    | Fsharp : target_runtime
    | Cuda : target_runtime
    | Rust : target_runtime
    | TypeScript : target_runtime
    | Python : target_runtime

### run_target_args'

In [ ]:
inl run_target_args' forall t u. (args : u) (fn : target -> (u -> t)) : t =
    backend_switch {
        Gleam = fun () =>
            inl target = Gleam Native
            fn target args
        Fsharp = fun () =>
            inl is_unit : bool =
                real
                    typecase t with
                    | () => true
                    | _ => false
            $'(* run_target_args\''
            inl result = $'()' : unit
            $'run_target_args\' *)'
            inl emit_result x : () =
                if is_unit |> not
                then $'let _run_target_args\'_!result = !x '
            $'\n#if FABLE_COMPILER || WASM || CONTRACT'
            $'\n#if FABLE_COMPILER_RUST && \!WASM && \!CONTRACT'
            inl target = Rust Native
            fn target args |> emit_result
            $'#endif\n#if FABLE_COMPILER_RUST && WASM'
            inl target = Rust Wasm
            fn target args |> emit_result
            $'#endif\n#if FABLE_COMPILER_RUST && CONTRACT'
            inl target = Rust Contract
            fn target args |> emit_result
            $'#endif\n#if FABLE_COMPILER_TYPESCRIPT'
            inl target = TypeScript Native
            fn target args |> emit_result
            $'#endif\n#if FABLE_COMPILER_PYTHON'
            inl target = Python Native
            fn target args |> emit_result
            $'#endif\n#if \!FABLE_COMPILER_RUST && \!FABLE_COMPILER_TYPESCRIPT && \!FABLE_COMPILER_PYTHON'
            inl target = Fsharp Wasm
            fn target args |> emit_result
            $'#endif\n#else'
            inl target = Fsharp Native
            fn target args |> emit_result
            $'#endif'
            if is_unit
            then $'// run_target_args\' is_unit'
            else $'_run_target_args\'_!result ' : t
        Python = fun () =>
            inl target = Cuda Native
            fn target args
    }

### run_target_args

In [ ]:
inl run_target_args forall t u. (args : () -> u) (fn : target -> (u -> t)) : t =
    inl args = args () |> dyn
    fn |> run_target_args' args

### run_target

In [ ]:
inl run_target forall t. (fn : target -> (() -> t)) : t =
    run_target_args id fn

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

run_target function
    | Gleam (Native) => fun () => $'1'
    | Fsharp (Native) => fun () => $'1uy'
    | Cuda (Native) => fun () => $'1'
    | Rust (Native) => fun () => $'1uy'
    | TypeScript (Native) => fun () => $'1uy'
    | Python (Native) => fun () => $'1uy'
    | _ => fun () => $'2uy'
|> _assert_eq 1u8

.py output (Cuda):
__assert_eq / actual: 1 / expected: 1

.rs output:
__assert_eq / actual: 1 / expected: 1

.ts output:
__assert_eq / actual: 1 / expected: 1

.py output:
__assert_eq / actual: 1 / expected: 1


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/60cf0654bee59a601f7f5bd49ba00a6948cf7506e54fea0d15eb95854818030c/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@float.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/60cf0654bee59a601f7f5bd49ba00a6948cf7506e54fea0d15eb95854818030c/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@order.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/60cf0654bee59a601f7f5bd49ba00a6948cf7506e54fea0d15eb95854818030c/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bool.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/60cf0654bee59a601f7f5bd49ba00a6948cf7506e54fea0d15eb95854818030c/build/dev/erlan

.fsx output:
__assert_eq / actual: 1uy / expected: 1uy


## function

### eval

In [ ]:
inl eval fn =
    fn ()

### flip

In [ ]:
inl flip fn a b =
    fn b a

### do

In [ ]:
inl do (body : () -> ()) : () =
    !!!!Do (body())

### indent

In [ ]:
inl indent (body : () -> ()) : () =
    backend_switch {
        Fsharp = fun () =>
            inl body () =
                body ()
                $'(* indent' : ()
            !!!!Indent (body())
            $'indent *)' : ()
        Python = fun () =>
            !!!!Indent (body())
            ()
    }

### let'

In [ ]:
inl let' fn =
    inl result : unit =
        backend_switch {
            Fsharp = fun () =>
                $'()' : unit
            Python = fun () =>
                $'None' : unit
        }
    backend_switch {
        Fsharp = fun () =>
            $'let _let\'_!result =' : ()
            fn |> indent
        Python = fun () =>
            $'def _let\'_!result():' : ()
            fn |> indent
    }
    $'_let\'_!result '

### exec_unit

In [ ]:
inl exec_unit (fn : () -> ()) : () =
    backend_switch {
        Fsharp = fun () =>
            inl unit = $'()' : $'unit'
            ($'(fun () -> !fn (); !unit) ()' : $'unit') |> ignore
        Python = fun () => fn ()
    }

### lazy

In [ ]:
nominal lazy t = $'Lazy<`t>'

### memoize

In [ ]:
nominal lazy t = $'Lazy<`t>'

inl memoize forall t. (fn : () -> t) : () -> t =
    inl fn = join fn
    inl body () =
        inl result = mut None
        inl computed = mut false
        fun () =>
            if *computed
            then *result
            else
                result <- fn () |> Some
                computed <- true
                *result
            |> optionm.value
    backend_switch {
        Gleam = fun () => body ()
        Fsharp = fun () =>
            inl result : lazy t = $'lazy !fn ()'
            fun () => $'!result.Value' : t
        Python = fun () => body ()
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

inl count = mut 0i32
inl add =
    fun () =>
        count <- *count + 1
        count
    |> memoize

add () |> ignore
add () |> ignore
add () |> ignore

*count
|> _assert_eq 1

.py output (Cuda):
__assert_eq / actual: 1 / expected: 1

.rs output:
__assert_eq / actual: 1 / expected: 1

.ts output:
__assert_eq / actual: 1 / expected: 1

.py output:
__assert_eq / actual: 1 / expected: 1



.fsx output:
__assert_eq / actual: 1 / expected: 1


### capture

In [ ]:
inl capture forall t. (fn : () -> t) : t =
    backend_switch {
        Fsharp = fun () =>
            inl result = dyn true
            $'let mutable _capture_!result : `t option = None '
            $'('
            $'(fun () ->'
            $'(fun () ->'
            fn () |> emit_unit
            $')'
            $'|> fun x -> x ()'
            $') () )'
            $'|> fun x -> _capture_!result <- Some x'
            $'match _capture_!result with Some x -> x | None -> failwith "base.capture / _capture_!result=None"' : t
        Python = fun () =>
            fn ()
    }

### yield_from

In [ ]:
inl yield_from forall (t : * -> *) u. (a : t u) : () =
    backend_switch {
        Fsharp = fun () => $'yield\! !a ' : ()
        Python = fun () => $'asyncio.run(!a())' : ()
    }

### join_body

In [ ]:
inl join_body body acc x =
    if var_is x |> not
    then body acc x
    else
        inl acc = dyn acc
        join body acc x

In [ ]:
//// test

inl rec fold_list f s = function
    | Cons (x, x') => fold_list f (f s x) x'
    | Nil => s

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python
//// print_code

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (+) 0
|> _assert_eq 15

.py output (Cuda):
__assert_eq / actual: 15 / expected: 15

.rs output:
__assert_eq / actual: 15 / expected: 15

.ts output:
__assert_eq / actual: 15 / expected: 15

.py output:
__assert_eq / actual: 15 / expected: 15


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/c3c91f2c77e9f90dd38a6a09d5b29eb2b2aee41b4b74a2c4dfc1b79f47d62dfb/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bit_array.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/c3c91f2c77e9f90dd38a6a09d5b29eb2b2aee41b4b74a2c4dfc1b79f47d62dfb/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bool.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/c3c91f2c77e9f90dd38a6a09d5b29eb2b2aee41b4b74a2c4dfc1b79f47d62dfb/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@pair.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/c3c91f2c77e9f90dd38a6a09d5b29eb2b2aee41b4b74a2c4dfc1b79f47d62dfb/buil

.gleam (Gleam):
import gleam/string
import gleam/io
pub fn method1 () -> Int {
    3
}
pub fn method2 (v0 :    Bool) -> Bool {
    v0
}
pub fn method0 () -> Nil   {
    let v0 = method1()
    let v1 = 9 + v0
    let v2 = v1 + 2
    let v3 = v2 + 1
    let v4 = v3 == 15
    let v6 =
        case v4 {
            True -> {
                True
            }
            False -> {
                method2(v4)
            }
        }
    let v7 = "__assert_eq"
    let v8 = string.inspect(v3)
    let v9 = string.inspect(15)
    let v10 = v7 <> " / actual: " <> v8 <> " / expected: " <> v9
    io.println(v10)
    let v18 = v6 == False
    case v18 {
        True -> {
            panic as v10
        }
        False -> {
        Nil
        }
    }
}
pub fn main () { method0()
 }
.fsx:
let rec method1 () : int32 =
    3
and method2 (v0 : bool) : bool =
    v0
and closure0 (v0 : string) () : unit =
    let v1 : (string -> unit) = System.Console.WriteLine
    v1 v0
and method0 () : unit =
    let

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python
//// print_code

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (join_body (+)) 0
|> _assert_eq 15

.py output (Cuda):
__assert_eq / actual: 15 / expected: 15

.rs output:
__assert_eq / actual: 15 / expected: 15

.ts output:
__assert_eq / actual: 15 / expected: 15

.py output:
__assert_eq / actual: 15 / expected: 15


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/c7ac0834504a9b8fef05cc6adf9ccd637888bd0650b2f960be38cdb6114d6a29/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bit_array.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/c7ac0834504a9b8fef05cc6adf9ccd637888bd0650b2f960be38cdb6114d6a29/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@result.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/c7ac0834504a9b8fef05cc6adf9ccd637888bd0650b2f960be38cdb6114d6a29/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam_stdlib_decode_ffi.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/c7ac0834504a9b8fef05cc6adf9ccd637888bd0650b2f960be38cd

.gleam (Gleam):
import gleam/string
import gleam/io
pub fn method1 () -> Int {
    3
}
pub fn method2 (v0 :    Int, v1 :    Int) -> Int {
    let v2 = v1 + v0
    v2
}
pub fn method3 (v0 :    Bool) -> Bool {
    v0
}
pub fn method0 () -> Nil   {
    let v0 = method1()
    let v1 = 9
    let v2 = method2(v0, v1)
    let v3 = v2 + 2
    let v4 = v3 + 1
    let v5 = v4 == 15
    let v7 =
        case v5 {
            True -> {
                True
            }
            False -> {
                method3(v5)
            }
        }
    let v8 = "__assert_eq"
    let v9 = string.inspect(v4)
    let v10 = string.inspect(15)
    let v11 = v8 <> " / actual: " <> v9 <> " / expected: " <> v10
    io.println(v11)
    let v19 = v7 == False
    case v19 {
        True -> {
            panic as v11
        }
        False -> {
        Nil
        }
    }
}
pub fn main () { method0()
 }
.fsx:
let rec method1 () : int32 =
    3
and method2 (v0 : int32, v1 : int32) : int32 =
    let v2 : int32 = v1

### join_body_unit

In [ ]:
inl join_body_unit body d x =
    if var_is d |> not
    then body x
    else
        inl x = dyn x
        join body x

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python
//// print_code

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (fun acc n => join_body_unit ((+) acc) n n) 0
|> _assert_eq 15

.py output (Cuda):
__assert_eq / actual: 15 / expected: 15

.rs output:
__assert_eq / actual: 15 / expected: 15

.ts output:
__assert_eq / actual: 15 / expected: 15

.py output:
__assert_eq / actual: 15 / expected: 15


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/a3d84dafa6a63bfa00c45858376bf72f1ca39e78a888e5500d56da580527583d/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bytes_tree.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/a3d84dafa6a63bfa00c45858376bf72f1ca39e78a888e5500d56da580527583d/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@int.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/a3d84dafa6a63bfa00c45858376bf72f1ca39e78a888e5500d56da580527583d/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@result.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/a3d84dafa6a63bfa00c45858376bf72f1ca39e78a888e5500d56da580527583d/bu

.gleam (Gleam):
import gleam/string
import gleam/io
pub fn method1 () -> Int {
    3
}
pub fn method2 (v0 :    Int) -> Int {
    let v1 = 9 + v0
    v1
}
pub fn method3 (v0 :    Bool) -> Bool {
    v0
}
pub fn method0 () -> Nil   {
    let v0 = method1()
    let v1 = method2(v0)
    let v2 = v1 + 2
    let v3 = v2 + 1
    let v4 = v3 == 15
    let v6 =
        case v4 {
            True -> {
                True
            }
            False -> {
                method3(v4)
            }
        }
    let v7 = "__assert_eq"
    let v8 = string.inspect(v3)
    let v9 = string.inspect(15)
    let v10 = v7 <> " / actual: " <> v8 <> " / expected: " <> v9
    io.println(v10)
    let v18 = v6 == False
    case v18 {
        True -> {
            panic as v10
        }
        False -> {
        Nil
        }
    }
}
pub fn main () { method0()
 }
.fsx:
let rec method1 () : int32 =
    3
and method2 (v0 : int32) : int32 =
    let v1 : int32 = 9 + v0
    v1
and method3 (v0 : bool) : bool =
  

## arithmetic

### (+.)

In [ ]:
inl (+.) forall t. (a : t) (b : t) : t =
    $'!a + !b '

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'3' : i32) +. ($'-6' : i32)
|> _assert_eq -3i32

.py output (Cuda):
__assert_eq / actual: -3 / expected: -3

.rs output:
__assert_eq / actual: -3 / expected: -3

.ts output:
__assert_eq / actual: -3 / expected: -3

.py output:
__assert_eq / actual: -3 / expected: -3


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/f4d10745928afa994c1c57c4fa927aa3eeccf27be6495b8f1fb4f3a3eeea48e1/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@result.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/f4d10745928afa994c1c57c4fa927aa3eeccf27be6495b8f1fb4f3a3eeea48e1/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@io.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/f4d10745928afa994c1c57c4fa927aa3eeccf27be6495b8f1fb4f3a3eeea48e1/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@string_tree.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/f4d10745928afa994c1c57c4fa927aa3eeccf27be6495b8f1fb4f3a3eeea48e1/bu

.fsx output:
__assert_eq / actual: -3 / expected: -3


### (-.)

In [ ]:
inl (-.) forall t. (a : t) (b : t) : t =
    $'!a - !b '

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'3' : i32) -. ($'6' : i32)
|> _assert_eq -3i32

.py output (Cuda):
__assert_eq / actual: -3 / expected: -3

.rs output:
__assert_eq / actual: -3 / expected: -3

.ts output:
__assert_eq / actual: -3 / expected: -3

.py output:
__assert_eq / actual: -3 / expected: -3


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/564412dd855228db656de9d06b4ee973ad969df939d72fa26d43996f3c34f192/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@set.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/564412dd855228db656de9d06b4ee973ad969df939d72fa26d43996f3c34f192/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam_stdlib.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/564412dd855228db656de9d06b4ee973ad969df939d72fa26d43996f3c34f192/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bool.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/564412dd855228db656de9d06b4ee973ad969df939d72fa26d43996f3c34f192/build/de

.fsx output:
__assert_eq / actual: -3 / expected: -3


### (*.)

In [ ]:
inl (*.) forall t. (a : t) (b : t) : t =
    $'!a * !b '

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'3' : i32) *. ($'-1' : i32)
|> _assert_eq -3i32

.py output (Cuda):
__assert_eq / actual: -3 / expected: -3

.rs output:
__assert_eq / actual: -3 / expected: -3

.ts output:
__assert_eq / actual: -3 / expected: -3

.py output:
__assert_eq / actual: -3 / expected: -3


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/662af073855223da1f7bc66e92199c9c7ce4614e6d1e02e0a807746c7f492a82/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@dynamic@decode.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/662af073855223da1f7bc66e92199c9c7ce4614e6d1e02e0a807746c7f492a82/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bytes_tree.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/662af073855223da1f7bc66e92199c9c7ce4614e6d1e02e0a807746c7f492a82/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@string.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/662af073855223da1f7bc66e92199c9c7ce4614e6d1e02e0a807746c

.fsx output:
__assert_eq / actual: -3 / expected: -3


### (/.)

In [ ]:
inl (/.) forall t. (a : t) (b : t) : t =
    $'!a / !b '

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'-3' : i32) /. ($'1' : i32)
|> _assert_eq -3i32

.py output (Cuda):
__assert_eq / actual: -3.0 / expected: -3

.rs output:
__assert_eq / actual: -3 / expected: -3

.ts output:
__assert_eq / actual: -3 / expected: -3

.py output:
__assert_eq / actual: -3 / expected: -3


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/d55a4ae3080bc59259ac69ecdb4cd1ac6a08f796896bc782f0dee02199085895/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@dynamic@decode.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/d55a4ae3080bc59259ac69ecdb4cd1ac6a08f796896bc782f0dee02199085895/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@order.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/d55a4ae3080bc59259ac69ecdb4cd1ac6a08f796896bc782f0dee02199085895/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bytes_tree.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/d55a4ae3080bc59259ac69ecdb4cd1ac6a08f796896bc782f0dee02

.fsx output:
__assert_eq / actual: -3 / expected: -3


## comparison

### (=.)

In [ ]:
inl (=.) forall t. (a : t) (b : t) : bool =
    backend_switch {
        Gleam = fun () => $'!a == !b ' : bool
        Fsharp = fun () => $'!a = !b ' : bool
        Python = fun () => $'!a == !b ' : bool
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'-3' : i32) =. ($'-3' : i32)
|> _assert_eq true

.py output (Cuda):
__assert_eq / actual: True / expected: True

.rs output:
__assert_eq / actual: true / expected: true

.ts output:
__assert_eq / actual: true / expected: true

.py output:
__assert_eq / actual: true / expected: true


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/f612567cf6494302ccaa99d14530ddb709370c80c48afeb8471575bff4edc966/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@set.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/f612567cf6494302ccaa99d14530ddb709370c80c48afeb8471575bff4edc966/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@function.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/f612567cf6494302ccaa99d14530ddb709370c80c48afeb8471575bff4edc966/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@dynamic.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/f612567cf6494302ccaa99d14530ddb709370c80c48afeb84715

.fsx output:
__assert_eq / actual: true / expected: true


### (<>.)

In [ ]:
inl (<>.) forall t. (a : t) (b : t) : bool =
    backend_switch {
        Gleam = fun () => $'!a \!= !b ' : bool
        Fsharp = fun () => $'!a <> !b ' : bool
        Python = fun () => $'!a \!= !b ' : bool
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'-3' : i32) <>. ($'3' : i32)
|> _assert_eq true

.py output (Cuda):
__assert_eq / actual: True / expected: True

.rs output:
__assert_eq / actual: true / expected: true

.ts output:
__assert_eq / actual: true / expected: true

.py output:
__assert_eq / actual: true / expected: true


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/1bfd32271987bcfb977f56d8dd1c3b6e39fdeeb9fcab3613725e1f1227137f5b/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@pair.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/1bfd32271987bcfb977f56d8dd1c3b6e39fdeeb9fcab3613725e1f1227137f5b/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@result.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/1bfd32271987bcfb977f56d8dd1c3b6e39fdeeb9fcab3613725e1f1227137f5b/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@dict.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/1bfd32271987bcfb977f56d8dd1c3b6e39fdeeb9fcab3613725e1f12

.fsx output:
__assert_eq / actual: true / expected: true


### (<>..)

In [ ]:
inl (<>..) a b =
    fun () => a = b
    |> dyn
    |> eval
    |> not

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'-3' : i32) <>.. ($'3' : i32)
|> _assert_eq true

.py output (Cuda):
__assert_eq / actual: True / expected: True

.rs output:
__assert_eq / actual: true / expected: true

.ts output:
__assert_eq / actual: true / expected: true

.py output:
__assert_eq / actual: true / expected: true


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/d6ef83fd597ed457a1525d86543e2a4e1297893b9351d12d6e664bcbeda78c8d/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bytes_tree.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/d6ef83fd597ed457a1525d86543e2a4e1297893b9351d12d6e664bcbeda78c8d/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@string_tree.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/d6ef83fd597ed457a1525d86543e2a4e1297893b9351d12d6e664bcbeda78c8d/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@function.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/d6ef83fd597ed457a1525d86543e2a4e1297893b9

.fsx output:
__assert_eq / actual: true / expected: true


## composition

### append

In [ ]:
prototype append t : t -> t -> t

### (++)

In [ ]:
inl (++) a b =
    b |> append a

## pair

### pair

In [ ]:
type pair_switch a b =
    {
        Gleam : $'#(`a, `b  )'
        Fsharp : $'(`a * `b)'
        Python : $'(`a, `b)'
    }
nominal pair a b = $'backend_switch `(pair_switch a b)'

inl pair x y =
    x, y

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

pair 1i32 2i32
|> _assert_eq (1, 2)

.py output (Cuda):
__assert_eq / actual: (1, 2) / expected: (1, 2)

.rs output:
__assert_eq / actual: (1, 2) / expected: (1, 2)

.ts output:
__assert_eq / actual: 1,2 / expected: 1,2

.py output:
__assert_eq / actual: (1, 2) / expected: (1, 2)


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/3b12d7c3f0448c8d76b2ceb9d762527c68839339d7efe0dc11833be8eb52e19d/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@function.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/3b12d7c3f0448c8d76b2ceb9d762527c68839339d7efe0dc11833be8eb52e19d/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam_stdlib_decode_ffi.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/3b12d7c3f0448c8d76b2ceb9d762527c68839339d7efe0dc11833be8eb52e19d/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@float.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/3b12d7c3f0448c8d76b2ceb9d76252

.fsx output:
__assert_eq / actual: struct (1, 2) / expected: struct (1, 2)


### new_pair

In [ ]:
inl new_pair forall a b. (a : a) (b : b) : pair a b =
    backend_switch {
        Gleam = fun () =>
            $'#(!a, !b )' : pair a b
        Fsharp = fun () =>
            $'!a, !b ' : pair a b
        Python = fun () =>
            $'!a, !b ' : pair a b
    }

### from_pair

In [ ]:
inl from_pair forall a b. (pair : pair a b) : a * b =
    backend_switch {
        Gleam = fun () =>
            $'let #(a, b) = !pair '
            ($'a' : a), ($'b' : b)
        Fsharp = fun () =>
            $'let (a, b) = !pair '
            ($'a' : a), ($'b' : b)
        Python = fun () =>
            $'a, b = !pair '
            ($'a' : a), ($'b' : b)
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

new_pair "a" (new_pair 1i32 "b")
|> from_pair
|> _assert_eq' ("a", (new_pair 1i32 "b"))

.py output (Cuda):
__assert_eq' / actual: ('a', (1, 'b')) / expected: ('a', (1, 'b'))

.rs output:
__assert_eq' / actual: ("a", (1, "b")) / expected: ("a", (1, "b"))

.ts output:
__assert_eq' / actual: a,1,b / expected: a,1,b

.py output:
__assert_eq' / actual: ('a', (1, 'b')) / expected: ('a', (1, 'b'))


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/913ce68ac6ec38072efc8a622945938745a518e9db350ddbf0b5940843bbe4ae/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam_stdlib.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/913ce68ac6ec38072efc8a622945938745a518e9db350ddbf0b5940843bbe4ae/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@int.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/913ce68ac6ec38072efc8a622945938745a518e9db350ddbf0b5940843bbe4ae/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@order.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spira

.fsx output:
__assert_eq' / actual: struct ("a", (1, "b")) / expected: struct ("a", (1, "b"))


## application

### (||>)

In [ ]:
inl (||>) (arg1, arg2) fn =
    arg2 |> fn arg1

### (||>)

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(3i32, 2i32)
||> fun a b => a - b
|> _assert_eq 1

.py output (Cuda):
__assert_eq / actual: 1 / expected: 1

.rs output:
__assert_eq / actual: 1 / expected: 1

.ts output:
__assert_eq / actual: 1 / expected: 1

.py output:
__assert_eq / actual: 1 / expected: 1


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/8d44dd6004c0c80bce9f64fbec49cd2f4d32944c5067143c8c44b757aee985a6/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bytes_tree.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/8d44dd6004c0c80bce9f64fbec49cd2f4d32944c5067143c8c44b757aee985a6/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@pair.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/8d44dd6004c0c80bce9f64fbec49cd2f4d32944c5067143c8c44b757aee985a6/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@result.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/8d44dd6004c0c80bce9f64fbec49cd2f4d32944c5067143c8c44b757aee985a6/build/dev

.fsx output:
__assert_eq / actual: 1 / expected: 1


In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(1i32, 2i32)
||> flip pair
|> _assert_eq (2, 1)

.py output (Cuda):
__assert_eq / actual: (2, 1) / expected: (2, 1)

.rs output:
__assert_eq / actual: (2, 1) / expected: (2, 1)

.ts output:
__assert_eq / actual: 2,1 / expected: 2,1

.py output:
__assert_eq / actual: (2, 1) / expected: (2, 1)


.gleam output (Gleam):
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/5f8392e72d772f85b4fd74994418c6c04fdfadd2cdd7dd7a838c00c16ba3f274/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@dynamic.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/5f8392e72d772f85b4fd74994418c6c04fdfadd2cdd7dd7a838c00c16ba3f274/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@string_tree.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/5f8392e72d772f85b4fd74994418c6c04fdfadd2cdd7dd7a838c00c16ba3f274/build/dev/erlang/gleam_stdlib/_gleam_artefacts/gleam@bit_array.erl"}
{compiling,"/home/runner/work/spiral/polyglot/target/spiral_Eval/packages/5f8392e72d772f85b4fd74994418c6c04

.fsx output:
__assert_eq / actual: struct (2, 1) / expected: struct (2, 1)


### fix_condition

In [ ]:
inl fix_condition x a b =
    if x ()
    then a () |> fun x => $'(* fix_condition then' : ()
    else
        $'fix_condition then *) else' : ()
        b () |> fun x => $'(* fix_condition else' : ()
    |> fun x => $'fix_condition else *)' : ()

## type

### infer

In [ ]:
nominal infer = $'_'

### infer'

In [ ]:
nominal infer' t = $'_'

### any

In [ ]:
nominal any = $'obj'

### null

In [ ]:
inl null forall t. () : t =
    backend_switch {
        Gleam = fun () => $'Nil' : t
        Fsharp = fun () => $'null |> unbox<`t>' : t
        Python = fun () => $'None' : t
    }

### defaultof

In [ ]:
inl defaultof forall t. () : t =
    $'Unchecked.defaultof<`t>'

### choice2'

In [ ]:
nominal choice2' a b = $'Choice<`a, `b>'

### choice2_unbox

In [ ]:
inl choice2_unbox forall t1 t2. (choice : choice2' t1 t2) : choice2 t1 t2 =
    run_target_args (fun () => choice) function
        | Fsharp _ => fun choice =>
            inl c1of2 (x : t1) : _ _ t2 = C1of2 x
            inl c2of2 (x : t2) : _ t1 _ = C2of2 x
            inl c1of2 = join c1of2
            inl c2of2 = join c2of2
            $'match !choice with Choice1Of2 x -> !c1of2 x | Choice2Of2 x -> !c2of2 x'
        | _ => fun _ => null ()

## ref

### ref

In [ ]:
nominal ref t = $'`t ref'

### new_ref

In [ ]:
inl new_ref forall t. (x : t) : ref t =
    $'ref !x '

### ref_value

In [ ]:
inl ref_value forall t. (x : ref t) : t =
    $'!x.Value'

### ref_set_value

In [ ]:
inl ref_set_value forall t. (value : t) (ref : ref t) : ref t =
    $'!ref.Value <- !value '
    ref

## convert

### to

In [ ]:
inl to forall t u. (x : t) : u =
    $'!x ' : u

### convert

In [ ]:
inl convert forall t u. (x : t) : u =
    backend_switch {
        Gleam = fun () => $'!x ' : u
        Fsharp = fun () => $'!x |> `u ' : u
        Python = fun () => $'`u(!x)' : u
    }

### unbox

In [ ]:
inl unbox forall t u. (x : t) : u =
    backend_switch {
        Gleam = fun () => x |> to : u
        Fsharp = fun () => $'!x |> unbox<`u>' : u
        Python = fun () => x |> to : u
    }

### u8

In [ ]:
inl u8 forall t. (x : t) : u8 =
    backend_switch {
        Gleam = fun () => x |> to : u8
        Fsharp = fun () => x |> $'uint8' : u8
        Python = fun () => x |> to : u8
    }

### u16

In [ ]:
inl u16 forall t. (x : t) : u16 =
    backend_switch {
        Gleam = fun () => x |> to : u16
        Fsharp = fun () => x |> $'uint16' : u16
        Python = fun () => $'!x & 0xFFFF' : u16
    }

### u64

In [ ]:
inl u64 forall t. (x : t) : u64 =
    backend_switch {
        Gleam = fun () => x |> to : u64
        Fsharp = fun () => x |> $'uint64' : u64
        Python = fun () => x |> to : u64
    }

### i32

In [ ]:
inl i32 forall t. (x : t) : i32 =
    backend_switch {
        Gleam = fun () => x |> convert : i32
        Fsharp = fun () => x |> convert : i32
        Python = fun () => x |> convert : i32
    }

### i64

In [ ]:
inl i64 forall t. (x : t) : i64 =
    backend_switch {
        Gleam = fun () => x |> to : i64
        Fsharp = fun () => x |> $'int64' : i64
        Python = fun () => x |> to : i64
    }

### f32

In [ ]:
inl f32 forall t. (x : t) : f32 =
    backend_switch {
        Gleam = fun () => x |> to : f32
        Fsharp = fun () => x |> $'float32' : f32
        Python = fun () => x |> to : f32
    }

### f64

In [ ]:
inl f64 forall t. (x : t) : f64 =
    backend_switch {
        Gleam = fun () => x |> to : f64
        Fsharp = fun () => x |> $'float' : f64
        Python = fun () => x |> to : f64
    }

### unativeint

In [ ]:
nominal unativeint = $'unativeint'

### convert_i32

In [ ]:
inl convert_i32 forall t. (x : t) : i32 =
    backend_switch {
        Gleam = fun () => x |> to : i32
        Fsharp = fun () => x |> $'System.Convert.ToInt32' : i32
        Python = fun () => x |> to : i32
    }

### convert_i32_base

In [ ]:
inl convert_i32_base forall t. (base : i32) (x : t) : i32 =
    backend_switch {
        Fsharp = fun () => $'System.Convert.ToInt32 (!x, !base)' : i32
        Python = fun () => $'int (!x, !base)' : i32
    }

### (:>)

In [ ]:
prototype (~:>) r : forall t. t -> r

### to_any

In [ ]:
inl to_any forall t. (obj : t) : any =
    obj |> to

### (~:>) any

In [ ]:
instance (~:>) any = to_any

## error

### exn

In [ ]:
nominal exn = $"backend_switch `({ Gleam : $'Nil'; Fsharp : $'exn'; Python : $'BaseException' })"

inl exn x =
    x |> $'`exn '

### try

In [ ]:
inl try forall t. (fn : () -> t) (ex_fn : exn -> option t) : option t =
    inl some x : option t = Some x
    inl some = dyn some
    inl fn = dyn fn
    inl ex_fn = dyn ex_fn
    backend_switch {
        Fsharp = fun () =>
            $'let result = ref !(None : option t)'
            $'try'
            $'    result.Value <- !fn () |> !some '
            $'with ex ->'
            $'    result.Value <- !ex_fn ex '
            $'result.Value' : option t
        Python = fun () =>
            $'result = !(None : option t)'
            inl fn = dyn fn
            inl ex_fn = dyn ex_fn
            $'try:'
            $'    result = !some(!fn())\n        \'\'\''
            $'\'\'\''
            $'except Exception as e:'
            $'    result = !ex_fn(e)'
            $'result' : option t
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

try
    fun () => a ;[ 0i32 ] |> am'.index 1i32 |> sm'.format
    (fun ex => $'!ex ' |> sm'.format_exception |> Some)
|> optionm.value
|> _assert_eq (run_target function
    | Gleam => fun () => null ()
    | Fsharp => fun () => join "System.IndexOutOfRangeException: Index was outside the bounds of the array."
    | Cuda => fun () => "index 1 is out of bounds for axis 0 with size 1"
    | Rust => fun () => "Exception { message: \"index out of bounds: the len is 1 but the index is 1\" }"
    | TypeScript => fun () => "Error: Index was outside the bounds of the array.\\nParameter name: index"
    | Python => fun () => "array index out of range"
)

.py output (Cuda):
__assert_eq / actual: index 1 is out of bounds for axis 0 with size 1 / expected: index 1 is out of bounds for axis 0 with size 1

.rs output:
__assert_eq / actual: "Exception { message: "index out of bounds: the len is 1 but the index is 1" }" / expected: "Exception { message: "index out of bounds: the len is 1 but the index is 1" }"

.ts output:
__assert_eq / actual: Error: Index was outside the bounds of the array.\nParameter name: index / expected: Error: Index was outside the bounds of the array.\nParameter name: index

.py output:
__assert_eq / actual: array index out of range / expected: array index out of range



.fsx output:
__assert_eq / actual: "System.IndexOutOfRangeException: Index was outside the bounds of the array." / expected: "System.IndexOutOfRangeException: Index was outside the bounds of the array."


### try_unit

In [ ]:
inl try_unit forall t. (fn : () -> ()) (ex_fn : (() -> exn) -> ()) : t =
    backend_switch {
        Fsharp = fun () => $'try' : ()
        Python = fun () => $'try:' : ()
    }
    fn |> indent
    backend_switch {
        Fsharp = fun () => $'with ex ->' : ()
        Python = fun () => $'except Exception as ex:' : ()
    }
    fun () =>
        inl ex = $'ex'
        inl ex () =
            ex
        ex_fn ex
    |> indent
    backend_switch {
        Fsharp = fun () =>
            $'(* try_unit'
            $'try_unit *)' : t
        Python = fun () => $'' : t
    }

### try_unit'

In [ ]:
inl try_unit' forall t. (ex_fn : (() -> exn) -> ()) (fn : () -> ()) : t =
    try_unit fn ex_fn

### try_finally

In [ ]:
inl try_finally forall t. (fn : () -> ()) (finally : () -> ()) : t =
    backend_switch {
        Fsharp = fun () => $'try' : ()
        Python = fun () => $'try:' : ()
    }
    fn |> indent
    backend_switch {
        Fsharp = fun () => $'finally' : ()
        Python = fun () => $'finally:' : ()
    }
    finally |> indent
    backend_switch {
        Fsharp = fun () =>
            $'(* try_finally'
            $'try_finally *)'
            ()
        Python = fun () => ()
    }